In [1]:
!pip install -q sentence-transformers ipywidgets

import pandas as pd
from sentence_transformers import SentenceTransformer, util
from IPython.display import display, Markdown
import ipywidgets as widgets
import torch
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 28.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 11.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 66.1 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pylibcugraph-cu12 24.12.0 requires pylibraft-cu12==24.12.*, but you have pylibraft-cu12 25.2.0 which is incompatible.
pylibcugraph-cu12 24.12.0 requires rmm-cu12==24.12.*, but you have 

2025-05-07 16:52:30.998941: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746636751.279195      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746636751.361320      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# Loading dataset
df = pd.read_csv("/kaggle/input/world-food-facts/en.openfoodfacts.org.products.tsv", 
                 sep='\t', low_memory=False)

# Keeping only numeric codes and required columns
df = df[['code', 'product_name', 'brands', 'ingredients_text', 
         'carbohydrates_100g', 'fat_100g', 'fiber_100g', 
         'proteins_100g', 'salt_100g', 'sugars_100g']].dropna(subset=['code', 'product_name'])

# Filtering to first 10,000 numeric product codes only
df = df[df['code'].apply(lambda x: str(x).isdigit())].head(10_000)

# Filling missing text
df['ingredients_text'] = df['ingredients_text'].fillna('')
df['product_name'] = df['product_name'].fillna('')
df['full_text'] = df['product_name'] + ' ' + df['ingredients_text']


In [3]:
model = SentenceTransformer('all-MiniLM-L6-v2')
product_embeddings = model.encode(df['full_text'].tolist(), convert_to_tensor=True)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/313 [00:00<?, ?it/s]

In [4]:
def search_products(query, top_k=5):
    query_embedding = model.encode(query, convert_to_tensor=True)
    hits = util.semantic_search(query_embedding, product_embeddings, top_k=top_k)[0]

    results = []
    for hit in hits:
        idx = hit['corpus_id']
        product = df.iloc[idx]
        code = product['code']
        name = product['product_name']
        ingredients = product['ingredients_text']
        results.append(f"- **{code}** – {name}\n  - *Ingredients*: {ingredients}")
    return results

In [5]:
query_box = widgets.Text(
    description='Query:',
    placeholder='e.g. low sugar cereal',
    layout=widgets.Layout(width='80%')
)

search_button = widgets.Button(description="Search")
output = widgets.Output()

def on_search_clicked(b):
    with output:
        output.clear_output()
        query = query_box.value.strip()
        if not query:
            print("❌ Please enter a query.")
            return
        
        results = search_products(query)
        display(Markdown(f"### 🔎 Top matches for query: *{query}*"))
        for r in results:
            display(Markdown(r))

search_button.on_click(on_search_clicked)

display(widgets.VBox([query_box, search_button, output]))